In [1]:
import sqlite3
import pandas as pd
import json
import re
from string import punctuation
from bs4 import BeautifulSoup
import os
from tqdm import tqdm

### Сбор таблицы entries

#### 1. Создание таблицы

In [34]:
con = sqlite3.connect("Dictionary_Final.db")
cur = con.cursor()

In [35]:
cur.execute('DROP TABLE entries')

In [36]:
cur.execute("""
CREATE TABLE IF NOT EXISTS entries (
    entry_id INTEGER PRIMARY KEY,
    edition INT, 
    entry_name TEXT,
    form TEXT,
    is_variant INT,
    variant_to_entry INT,
    gram_pos TEXT,
    gram_gen TEXT,
    gram_degree TEXT,
    gram_number TEXT,
    gram_aspect TEXT,
    gram_itype TEXT,
    reversed TEXT,
    id_entry_to_see INT,
    idiom_id INT,
    full_text TEXT
)
""")

#### 2. Сбор всех json с информацией. Сортировка для того, чтобы сохранить порядок томов.

In [37]:
list_of_files = []
path = './all_entries'
for (dirpath, dirnames, filenames) in os.walk(path):
    for filename in filenames:
        if filename.endswith('json'):
            list_of_files.append(os.path.join(dirpath, filename))

In [38]:
new_list = []
for f in list_of_files:
    edition = f.split('.')[-2].split('_')[-1]
    new_list.append((int(edition), f))
new_list.sort(key=lambda x:x[0])

In [39]:
# функция, которая изменяет варианты типа CЪ-
# пример: СОВНЕСТИ - СЪ- -> СОВНЕСТИ - СЪВНЕСТИ
def change_variant(word, variant):
    new_variant = variant
    if variant.startswith('-'):
        variant = variant.strip('-')
        if variant == 'СЯ':
            new_variant = word+variant
        else:
            to_change = len(variant)
            new_variant = word[:-to_change] + variant
    elif variant.endswith('-'):
        variant = variant.strip('-')
        to_change = len(variant)
        new_variant = variant + word[to_change:]
    return new_variant

In [40]:
# сбор всех слов с их id, чтобы сделать правильные ссылки
# от одного слова к другому
word_ids = {}
i = 1
for filename in new_list:
    path = filename[1]
    with open(path) as f:
        words = json.load(f)
        for entry in words:
            if len(entry) > 0:
                word = entry['word'].replace(' ', '').lower()
                if word not in word_ids.keys():
                    word_ids[word] = i
                # если у слова есть вариативность, тоже записываем
                if entry['variant']:
                        j = i
                        for var in entry['variant']:
                            j += 1
                            if '-' in var:
                                var = change_variant(entry['word'], var)
                            if var.lower() not in word_ids.keys():
                                word_ids[var.lower()] = j
                        i = j
                i += 1

#### 3. Запись в базу данных, сохранение информации о значениях и примерах

In [41]:
i = 1
definitions = []  # значения и примеры
ref_to_find = []  # reference, которые не удалось обработать
for filename in new_list:
    edition = filename[0]
    path = filename[1]
    with open(path) as f:
        words = json.load(f)
        for entry in words:
            if len(entry) > 0:
                reversed_word = entry['word'][::-1]
                entry['word'] = entry['word'].replace(' ', '')
                if 'number' not in entry.keys():
                    entry['number'] = None
                if 'ref' not in entry.keys():
                    entry['ref'] = None
                if 'aspect' not in entry.keys():
                    entry['aspect'] = None
                if 'iType' not in entry.keys():
                    entry['iType'] = None
                if '1sg' not in entry.keys():
                    entry['1sg'] = None
                if 'pos' not in entry.keys():
                    entry['pos'] = None
                if (not entry['pos']) and (entry['gen']):
                    entry['pos'] = 'NOUN'
                if isinstance(entry['1sg'], list):
                    entry['1sg'] = ','.join(entry['1sg'])
                if isinstance(entry['pos'], list):
                    entry['pos'] = ','.join(entry['pos'])
                if entry['ref'] and isinstance(entry['ref'], list):
                    refs = []
                    for ref in entry['ref']:
                        if '(' in ref:
                            ref = ref.split()[0]
                        if ref.lower() in word_ids.keys():
                            ref = word_ids[ref.lower()]
                        elif f"{ref.lower()}_1" in word_ids.keys():
                            ref = word_ids[f"{ref.lower()}_1"]
                        else:
                            ref_to_find.append((entry['word'], ref))
                        refs.append(str(ref))
                    entry['ref'] = ','.join(refs)
                cur.execute(
                    'INSERT INTO entries VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',
                    (i, edition, entry['word'], entry['1sg'], 0, None, entry['pos'], 
                    entry['gen'], entry['degree'], entry['number'], entry['aspect'],
                    entry['iType'], reversed_word, entry['ref'], None, entry['full_text'])
                )
                if entry['senses']:
                    for key in entry['senses'].keys():
                        definitions.append({
                            'word_id': i, 
                            'edition': edition,
                            'definition': key, 
                            'examples': entry['senses'][key]
                        })
                if entry['variant']:
                    j = i
                    for var in entry['variant']:
                        if '-' in var:
                            var = change_variant(entry['word'], var)
                        j += 1
                        reversed_word = var[::-1]
                        cur.execute(
                            'INSERT INTO entries VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',
                            (j, edition, var, None, 1, i, None, None, None, 
                            None, None, None, reversed_word, None, None, None)
                        )
                    i = j
                i += 1

                con.commit()

In [42]:
definitions[0]

{'word_id': 1,
 'edition': 1,
 'definition': "Буква русской азбуки, называемая аз; числовое значение' — 1 (·а҃·), со знаком ҂ 1000.",
 'examples': [['Въ ·а҃·-вую недѣлю.',
   'Ж.Феодос. Нест.',
   '77',
   'XII в.',
   '~ 1088 г.'],
  ['Коли ся грамота псана ишлъ былъ от рж҃ства гс҃ня до сего лѣта ·҂а҃· лѣт҃ и ·с҃· лѣт҃ и ·и҃· лѣт҃ и ·к·.',
   'Смол.гр.',
   '24',
   '1229 г.',
   None]]}

### Значения и примеры

#### 1. Сбор информации из таблицы по источникам и добавление в итоговую

In [11]:
con = sqlite3.connect("Sources.db")
cur = con.cursor()

In [44]:
cur.execute('SELECT * FROM sources')
sources = cur.fetchall()

In [15]:
cur.execute('SELECT * FROM languages')
languages = cur.fetchall()

In [16]:
cur.execute('SELECT * FROM sources_types')
sources_types = cur.fetchall()

In [17]:
con = sqlite3.connect("Dictionary_Final.db")
cur = con.cursor()

In [143]:
#cur.execute('DROP TABLE sources_types')
#cur.execute('DROP TABLE languages')
#cur.execute('DROP TABLE sources')

In [19]:
cur.execute("""
CREATE TABLE IF NOT EXISTS sources_types (
    type_id INTEGER PRIMARY KEY,
    type TEXT
)
""")
cur.executemany('INSERT INTO sources_types VALUES (?, ?)', sources_types)
con.commit()

In [21]:
cur.execute("""
CREATE TABLE IF NOT EXISTS sources (
    source_id INTEGER PRIMARY KEY,
    full TEXT,
    abbr_name TEXT,
    lang_id INT,
    date_start_y INT,
    date_start_c INT,
    date_end_y INT,
    date_end_c INT,
    publication_date_y INT,
    publication_date_c INT,
    type_id INT,
    is_translated INT
)
""")
cur.executemany(
    'INSERT INTO sources VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', 
    sources
)
con.commit()

#### 2. Сбор id источников для каждого примера

Формат ключа _examples_:
1. Пример
2. Источник (id)
3. Страница источника
4. Дата памятника, по которому происходит цитирование
5. Период первой засвидетельствованной или предполагаемой версии памятника (если памятник существует во множестве списков разного времени)

In [56]:
print(definitions[0])

{'word_id': 1, 'edition': 1, 'definition': "Буква русской азбуки, называемая аз; числовое значение' — 1 (·а҃·), со знаком ҂ 1000.", 'examples': [['Въ ·а҃·-вую недѣлю.', 'Ж. Феодос. Нест.', '77', 'XII в.', '~ 1088 г.'], ['Коли ся грамота псана ишлъ былъ от рж҃ства гс҃ня до сего лѣта ·҂а҃· лѣт҃ и ·с҃· лѣт҃ и ·и҃· лѣт҃ и ·к·.', 'Смол. гр.', '24', '1229 г.', None]]}


In [45]:
for definition in tqdm(definitions):
    for example in definition['examples']:
        for source in sources:
            try:
                ed = example[1].replace(' ', '')
                if ed in source[1] or ed in source[2]:
                    example[1] = source[0]
                    break
            except:
                if ed in source[1] or ed in source[2]:
                    example[1] = source[0]
                    break

100%|██████████| 88630/88630 [00:41<00:00, 2118.20it/s]


In [46]:
print(definitions[0])

{'word_id': 1, 'edition': 1, 'definition': "Буква русской азбуки, называемая аз; числовое значение' — 1 (·а҃·), со знаком ҂ 1000.", 'examples': [['Въ ·а҃·-вую недѣлю.', 488, '77', 'XII в.', '~ 1088 г.'], ['Коли ся грамота псана ишлъ былъ от рж҃ства гс҃ня до сего лѣта ·҂а҃· лѣт҃ и ·с҃· лѣт҃ и ·и҃· лѣт҃ и ·к·.', 1218, '24', '1229 г.', None]]}


In [47]:
for example in definitions[0]['examples']:
    print(example[1])

488
1218


In [48]:
cur.execute('DROP TABLE quote_to_definition')
cur.execute('DROP TABLE definitions')
cur.execute('DROP TABLE quotes')

In [49]:
cur.execute("""
CREATE TABLE IF NOT EXISTS quote_to_definition (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    quote_id INT,
    definition_id INT
)
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS definitions (
    definition_id INTEGER PRIMARY KEY,
    entry_id INT, 
    text_meaning TEXT, 
    idiom_id INT, 
    related_entries TEXT
)
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS quotes (
    quote_id INTEGER PRIMARY KEY,
    source_id INT,
    quote TEXT, 
    page INT,
    edition INT,
    base_dict_id INT,
    orig_date_start_y INT,
    orig_date_start_c TEXT,
    orig_date_end_y INT,
    orig_date_end_c TEXT,
    date_start_y INT,
    date_start_c TEXT,
    date_end_y INT,
    date_end_c TEXT
)
""")

In [50]:
# функция, чтобы перевести значение века в числовое
mistakes = []
def get_century(century):
    century = century.replace('\n\n\n\n', '')
    century_to_num = {
        'I': 1, 'II': 2, 'III': 3, 'IIІ': 3, 'IV': 4, 'IѴ': 4,
        'V': 5, 'VI': 6, 'VII': 7, 'VIII': 8, 'IX': 9, 'X': 10, 
        'XI': 11, 'XІ': 11, 'ΧI': 11, 'XII': 12, 'XІІ': 12, 'XIІ': 12, 
        'XIІІ': 13, 'XІІІ': 13, 'XIII': 13, 'XІV': 14, 'XIV': 14,
        'XV': 15, 'XVI': 16, 'ΧVΙ': 16, 'XVІ':16, 
        'XVII': 17, 'XVIІ':17, 'XVІI': 17, 'XVIΙ':17, 'XVІІ':17, 
        'XVІІІ': 18, 'XVIII':18, 'XVІII': 18, 'XIX': 19, 'XX': 20, 'XIX': 21, '': ''
    }
    if '—' in century:
        centuries = century.split('—')
    elif '–' in century:
        centuries = century.split('–')
    else:
        centuries = century.split('-')
    new_nums = []
    try:
        for c in centuries:
            new_nums.append(str(century_to_num[c]))
    except:
        new_nums = centuries
        mistakes.append(centuries[0])
    return new_nums

In [51]:
# функция, чтобы оставить только год, век или промежуток
def clean_year(year):
    year = year.replace('или', '-')
    year = year.replace('и', '-')
    wo_letters = re.sub(r'[а-яА-ЯёЁ]', '', year)
    clean_year = wo_letters.\
                replace('~', '').\
                replace(' ', '').\
                strip(',()').\
                replace('.', '').\
                replace('?', '').\
                replace('(', '').\
                replace('——', '-')
    return clean_year

In [52]:
problems = []  # список дат, которые не удалось перевести в нужный формат
# функция, которая переводит даты (век) в числовой, разбирает промежутки на дату начала и конца
def get_dates(date):
    date_start_y, date_start_c, date_end_y, date_end_c = None, None, None, None
    # проверка, если дата - это век
    if ('в' in date) or ('X' in date):
        century = get_century(clean_year(date))  # очистка и получение числового значения
        if len(century) == 1:  # если значение одно, то начало и конец одинаковые
            date_start_c = century[0]
            date_end_c = century[0]
        else:
            date_start_c = century[0]
            date_end_c = century[1]
    else: 
        year = clean_year(date)
        year = year.replace('-', '—')  # всего встречается три разных тире
        year = year.replace('–', '—')  # я их заменяю на обычный, чтобы было проще
        try:
            if '—' in year:  # если есть тире -> перед нами промежуток
                nums = year.split('—')
                date_start_y, date_end_y = nums[0], nums[1]
                date_start_c = int(date_start_y[:-2]) + 1
                if len(date_end_y) > 2:
                    date_end_c = int(date_end_y[:-2]) + 1
                # если второе число имеет 2 символа, скорее всего это промежуток вида 1467-90
                # поэтому век такой же, как у начала
                else:  
                    date_end_c = date_start_c
            else:
                date_start_y = date_end_y = year
                if len(year) > 2:
                    date_start_c = date_end_c = int(year[:-2]) + 1
                else:
                    date_start_c = date_end_c = 1
        except:
            problems.append(example)
    return date_start_y, date_start_c, date_end_y, date_end_c

In [53]:
j = 1  
quotes = {}  # список цитат, чтобы не было повторений
for i, definition in enumerate(definitions):
    cur.execute(
        'INSERT INTO definitions VALUES (?, ?, ?, ?, ?)',
        (i, definition['word_id'], definition['definition'], None, None)
    )
    for example in definition['examples']:
        orig_date_start_y, orig_date_start_c, orig_date_end_y, orig_date_end_c = None, None, None, None
        date_start_y, date_start_c, date_end_y, date_end_c = None, None, None, None
        if example[3]:
            date_start_y, date_start_c, date_end_y, date_end_c = get_dates(example[3])

        if example[4]:
            orig_date_start_y, orig_date_start_c, orig_date_end_y, orig_date_end_c = get_dates(example[4])
        
        if example[0] not in quotes:  # проверка, чтобы пример ещё не был в таблице
            quotes[example[0]] = j
            cur.execute(
                'INSERT INTO quotes VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                (j, example[1], example[0], example[2], definition['edition'],
                None, orig_date_start_y, orig_date_start_c, orig_date_end_y, orig_date_end_c,
                date_start_y, date_start_c, date_end_y, date_end_c)
            )
            j += 1

        cur.execute(
                'INSERT INTO quote_to_definition (quote_id, definition_id) VALUES (?, ?)',
                (quotes[example[0]], i)
            )
    con.commit()

### Этимология и глоссы

#### 1. Загрузка таблиц с глоссами и этимологиями

Также добавление языков в отдельную таблицу.

In [54]:
etym = pd.read_csv('etymology.csv', index_col=0)
gloss = pd.read_csv('glosses.csv', index_col=0)

In [55]:
# сбор id слов
def get_ids(lemma):
    if lemma.lower() in word_ids.keys():
        return word_ids[lemma.lower()]
    elif f"{lemma.lower()}_1" in word_ids.keys():
        return word_ids[f"{lemma.lower()}_1"]
    return lemma
gloss['word_id'] = gloss['lemma'].apply(get_ids)
etym['word_id'] = etym['lemma'].apply(get_ids)

In [56]:
etym['lang'] = etym['lang'].replace({"ukr' rend=": 'ukr'})
gloss['lang'] = gloss['lang'].replace({"ukr' rend=": 'ukr'})

In [57]:
langs = set(gloss['lang']) | set(etym['lang'])

In [58]:
languages = {'grk': 0, 'lat': 1, 'deu': 2, 'ces': 3}
i = 4
for l in langs:
    if l not in languages and isinstance(l, str):
        languages[l] = i
        i += 1

In [59]:
languages

{'grk': 0,
 'lat': 1,
 'deu': 2,
 'ces': 3,
 'gmh': 4,
 'hun': 5,
 'ang': 6,
 'kaz': 7,
 'lat-mid': 8,
 'ara': 9,
 'swe': 10,
 'nld': 11,
 'bul': 12,
 'pol-reg': 13,
 'san': 14,
 'gml': 15,
 'dan': 16,
 'ell': 17,
 'nor': 18,
 'sla': 19,
 'tur': 20,
 'lat/deu': 21,
 'fra': 22,
 'got': 23,
 'crh': 24,
 'grc': 25,
 'pol-old': 26,
 'uzb': 27,
 'fin': 28,
 'rus-new/ukr': 29,
 'bel-old': 30,
 'slv': 31,
 'qwm': 32,
 'syd': 33,
 'lit': 34,
 'dsb': 35,
 'ita': 36,
 'pol': 37,
 'hbs': 38,
 'gmh/pol': 39,
 'srp': 40,
 'trk': 41,
 'krl': 42,
 'grc/lat': 43,
 'goh': 44,
 'grc/mixed': 45,
 'chg': 46,
 'bul-mid': 47,
 'eng': 48,
 'rus/ukr': 49,
 'arc': 50,
 'chu': 51,
 'fro': 52,
 'tat': 53,
 'mon': 54,
 'chv': 55,
 'rus': 56,
 'nld/deu': 57,
 'fas': 58,
 'nds': 59,
 'mdf': 60,
 'ukr': 61,
 'aze': 62,
 'bel': 63,
 'odt': 64,
 'far': 65,
 'hbo': 66,
 'oss': 67,
 'grc/hbo': 68,
 'vep': 69}

In [ ]:
cur.execute("""
CREATE TABLE IF NOT EXISTS languages (
    lang_id INTEGER PRIMARY KEY,
    language TEXT
)
""")
cur.executemany('INSERT INTO languages VALUES (?, ?)', list(languages.items()))
con.commit()

In [60]:
languages_reverse = {i: w for w, i in languages.items()}

In [61]:
etym['lang'] = etym['lang'].map(languages)
gloss['lang'] = gloss['lang'].map(languages)

In [62]:
gloss

,edition,lemma,etym_lemma,lang,word_id
0,1,аистъ,bocianow,37.0,91
1,1,алкионитский,ἀλκυονίδας τὰς ἡμέρας,25.0,алкионитский
2,1,ангеловецъ,περὶ ἀγγελίτων,25.0,306
3,1,анкира,ἄγκυρα,25.0,328
4,1,апоновь,ἐπὶ ἐσχάρας,25.0,365
...,...,...,...,...,...
33601,9,мятежелюбство,νεωτεροποιΐαν,25.0,35114
33602,9,мятежетворецъ,στασιασταί,25.0,35116
33603,9,мятение,ταραχή,25.0,35131
33604,9,мятение,καθ’ ὕπαρ φαντάσματα,25.0,35131


In [63]:
etym_values = etym[['word_id', 'etym_lemma', 'lang']].values
gloss_values = gloss[['word_id', 'etym_lemma', 'lang']].values

In [64]:
cur.execute('DROP TABLE etymology')
cur.execute('DROP TABLE glosses')

In [65]:
cur.execute("""
CREATE TABLE IF NOT EXISTS etymology (
    etym_id INTEGER PRIMARY KEY, 
    word_id INT,
    etym_lemma TEXT,
    language TEXT
)
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS glosses (
    gloss_id INTEGER PRIMARY KEY, 
    word_id INT,
    gloss_lemma TEXT,
    language TEXT
)
""")

In [66]:
for i, values in enumerate(etym_values):
    cur.execute(
        'INSERT INTO etymology VALUES (?,?,?,?)',
        (i, values[0], values[1], values[2]))
    con.commit()

for i, values in enumerate(gloss_values):
    cur.execute(
        'INSERT INTO glosses VALUES (?,?,?,?)',
        (i, values[0], values[1].strip('()'), values[2]))
    con.commit()

#### Код ниже - код, по которому собирались таблицы с этимологиями и глоссами (чтение тегов tei)

In [159]:
etym = pd.read_csv('etym.tsv', sep='\t', index_col=0)

In [160]:
etym

,выпуск,лемма,старый тег,новый тег
1,,,,
2,1,актазъ,"<etym lang=""tur"">Ср. тюрк. <ref type=""lemma"" r...","<etym lang=""trk/fas""><lbl type=""cf"">Ср.</lbl> ..."
3,1,анкира,"<etym lang=""grc"">(<lbl type=""cf"">ср.</lbl> гре...","<etym lang=""grc"">(<lbl type=""cf"">ср.</lbl> <la..."
4,1,аргументовати,"<etym lang=""pl"">(<lbl type=""cf"">Ср.</lbl> поль...","<etym lang=""pol"">(<lbl type=""cf"">Ср.</lbl> <la..."
5,1,ариевъ,"<etym lang=""grc"">(От<lb/>неточного перевода гр...","<etym lang=""grc"">(<lbl type=""mistranslation"">О..."
6,1,армата,"<etym lang=""pl"">(<lbl type=""cf"">Ср.</lbl> поль...","<etym lang=""pol"">(<lbl type=""cf"">Ср.</lbl> <la..."
...,...,...,...,...
1316,27,станъ,"<etym lang=""grc"">греч. <ref type=""lemma"" lang=...","<etym lang=""grc""><lang>греч.</lang> <ref type=..."
1317,27,стандартъ,"<etym lang=""ger"">(<lbl type=""cf"">ср.</lbl><lb/...","<etym lang=""deu"">(<lbl type=""cf"">ср.</lbl><lb/..."
1318,27,становиско,"<etym lang=""cz/pl"">(<lbl type=""cf"">ср.</lbl> п...","<etym lang=""pol"">(<lbl type=""cf"">ср.</lbl> <la..."


In [167]:
def cool_lemma(lemma):
    if lemma.endswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')):
        return f'{lemma[:-1]}_{lemma[-1]}'
    return lemma


In [170]:
new_etym = pd.DataFrame(columns=['edition', 'lemma', 'etym_lemma', 'lang'])
for i, row in etym.iterrows():
    soup = BeautifulSoup(row['новый тег'])
    lemmas = soup.find_all('ref')
    ru_lemma = cool_lemma(row['лемма'])
    for lemma in lemmas:
        try:
            lemma_lang = lemma.attrs['lang']
            word = lemma.text.strip('().,:""«»')
            new_etym = new_etym.append({'edition': row['выпуск'], 'lemma': ru_lemma, 'etym_lemma': word, 'lang': lemma_lang}, ignore_index=True)
        except:
            try:
                lemma_lang = soup.find('etym').attrs['lang']
                word = lemma.text.strip('().,:""«»')
                new_etym = new_etym.append({'edition': row['выпуск'], 'lemma': ru_lemma, 'etym_lemma': word, 'lang': lemma_lang}, ignore_index=True)
            except:
                print(soup)

<etym>(<lbl type="meaning">от</lbl> <ref class="def" type="lemma">санъ</ref><note>?</note>)</etym>
<etym>(<lbl type="cf">ср.</lbl> <ref type="lemma">синаксарь</ref>)</etym>


In [175]:
new_etym[25:35]

,edition,lemma,etym_lemma,lang
25,3,вополчатися,παρατάττεται,grc
26,3,вополчатися,exponitur,lat
27,3,выдухати,έμφυςῶῡ,grc
28,3,выдухати,insufflans,lat
29,3,выправити,wyprawiс́,pol
30,3,выходъ,харадж,ara
31,3,выходъ,хараджа,ara
32,4,гажий,gad,pol
33,4,гапелька,гаплик,ukr
34,4,гафуница,haufnicĕ,ces


In [176]:
new_etym.to_csv('etymology.csv')

In [177]:
gloss = pd.read_csv('gloss.tsv', sep='\t', index_col=0)

In [99]:
'vffvf1'.endswith(('1', '2', '3'))

True

In [178]:
new_gloss = pd.DataFrame(columns=['edition', 'lemma', 'etym_lemma', 'lang'])
for i, row in gloss.iterrows():
    soup = BeautifulSoup(row['новый тег'])
    ru_lemma = cool_lemma(row['лемма'])
    try:
        lang = soup.find('gloss').attrs['lang']
    except:
        lang = None
    try:
        lemma = soup.find('ref').text.strip('().,:""«»')
    except:
        try:
            lemma = soup.find('gloss').text.strip('().,:""«»')
        except:
            pass
    new_gloss = new_gloss.append({'edition': row['выпуск'], 'lemma': ru_lemma, 'etym_lemma': lemma, 'lang': lang}, ignore_index=True)

In [179]:
new_gloss

,edition,lemma,etym_lemma,lang
0,1,аистъ,bocianow,pol
1,1,алкионитский,ἀλκυονίδας τὰς ἡμέρας,grc
2,1,ангеловецъ,περὶ ἀγγελίτων,grc
3,1,анкира,ἄγκυρα,grc
4,1,апоновь,ἐπὶ ἐσχάρας,grc
...,...,...,...,...
33601,9,мятежелюбство,νεωτεροποιΐαν,grc
33602,9,мятежетворецъ,στασιασταί,grc
33603,9,мятение,ταραχή,grc
33604,9,мятение,καθ’ ὕπαρ φαντάσματα,grc


In [90]:
gloss

,выпуск,лемма,старый тег,новый тег
1,,,,
2,1,аистъ,"<gloss lang=""pl"" class=""ex"">(bocianow)</gloss>","<gloss lang=""pol"" class=""ex"">(bocianow)</gloss>"
3,1,алкионитский,"<gloss lang=""grc"" class=""ex"">(ἀλκυονίδας τὰς ἡ...","<gloss lang=""grc"" class=""ex"">(ἀλκυονίδας τὰς ἡ..."
4,1,ангеловецъ,"<gloss lang=""grc"" class=""ex"">(περὶ ἀγγελίτων)<...","<gloss lang=""grc"" class=""ex"">(περὶ ἀγγελίτων)<..."
5,1,анкира,"<gloss lang=""grc"" class=""ex"">(ἄγκυρα)</gloss>","<gloss lang=""grc"" class=""ex"">(ἄγκυρα)</gloss>"
6,1,апоновь,"<gloss lang=""grc"" class=""ex"">(ἐπὶ ἐσχάρας)</gl...","<gloss lang=""grc"" class=""ex"">(ἐπὶ ἐσχάρας)</gl..."
...,...,...,...,...
33603,9,мятежелюбство,"<gloss lang=""grc"" class=""ex"">(νεωτεροποιΐαν)</...","<gloss lang=""grc"" class=""ex"">(νεωτεροποιΐαν)</..."
33604,9,мятежетворецъ,"<gloss lang=""grc"" class=""ex"">(στασιασταί)</gloss>","<gloss lang=""grc"" class=""ex"">(στασιασταί)</gloss>"
33605,9,мятение,"<gloss lang=""grc"" class=""ex"">(ταραχή)</gloss>","<gloss lang=""grc"" class=""ex"">(ταραχή)</gloss>"


In [10]:
gloss

,edition,lemma,gloss,lang
1,,,,
2,1,аистъ,(bocianow),pl
3,1,алкионитский,(ἀλκυονίδας τὰς ἡμέρας),grc
4,1,ангеловецъ,(περὶ ἀγγελίτων),grc
5,1,анкира,(ἄγκυρα),grc
6,1,апоновь,(ἐπὶ ἐσχάρας),grc
...,...,...,...,...
30471,27,старинствовати,(ἡγεμονεύοντος),grc
30472,27,старица,(μία γραῖα),grc
30473,27,старица,(εἰς...γραιΐδα),grc


In [180]:
new_gloss.to_csv('glosses.csv')